# Finite Domain Ewald Sphere Broadening

This tutorial demonstrates how finite coherent domain size affects RHEED diffraction patterns. Real crystal surfaces are never perfectly ordered over infinite distances - defects, domain boundaries, and limited terrace sizes restrict the coherent scattering length.

## Physical Background

In kinematic diffraction, we typically assume the Ewald sphere intersects infinitely sharp reciprocal lattice rods. In reality:

1. **Finite domain size** causes reciprocal space broadening: rod width σ ∝ 1/L
2. **Beam energy spread** and **divergence** create a finite Ewald "shell" thickness
3. The measured intensity is the **overlap integral** between broadened rods and shell

This tutorial covers:
- Computing rod widths from domain extent
- Calculating Ewald shell thickness from beam parameters
- Visualizing the rod-Ewald overlap
- Comparing RHEED patterns for different domain sizes

In [ ]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import rheedium as rh

# Suppress JAX GPU warning if no CUDA
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

## 1. Load Crystal Structure

We'll use MgO as our example - a simple rock-salt structure.

In [ ]:
crystal = rh.inout.parse_cif("../tests/test_data/MgO.cif")

print(f"Cell parameters: a = {float(crystal.cell_lengths[0]):.3f} Å")
print(f"Number of atoms: {crystal.cart_positions.shape[0]}")
print(f"Atomic positions (fractional):")
print(crystal.frac_positions)

## 2. Rod Width from Domain Size

The reciprocal lattice rod width is inversely proportional to domain size:

$$\sigma_q = \frac{2\pi}{L \times \sqrt{2\pi}}$$

This ensures the Gaussian approximation has the same FWHM as the true sinc² profile.

In [ ]:
# Compute rod widths for different domain sizes
domain_sizes = jnp.logspace(1, 3, 50)  # 10 to 1000 Å

rod_sigmas = []
for L in domain_sizes:
    extent = jnp.array([L, L, L / 2])  # Typical thin film: Lx = Ly, Lz smaller
    sigma = rh.simul.extent_to_rod_sigma(extent)
    rod_sigmas.append(float(sigma[0]))  # x-component

rod_sigmas = jnp.array(rod_sigmas)

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(domain_sizes, rod_sigmas, "b-", linewidth=2)
ax.set_xlabel("Domain size L (Å)", fontsize=12)
ax.set_ylabel("Rod width σ (Å⁻¹)", fontsize=12)
ax.set_title("Reciprocal Lattice Rod Broadening", fontsize=14)
ax.grid(True, alpha=0.3)

# Add reference lines
ax.axhline(y=0.1, color="r", linestyle="--", alpha=0.5, label="σ = 0.1 Å⁻¹")
ax.axhline(y=0.01, color="g", linestyle="--", alpha=0.5, label="σ = 0.01 Å⁻¹")
ax.legend()

plt.tight_layout()
plt.show()

# Print some values
for L in [20, 50, 100, 500]:
    extent = jnp.array([float(L), float(L), float(L) / 2])
    sigma = rh.simul.extent_to_rod_sigma(extent)
    print(f"L = {L:4d} Å → σ = {float(sigma[0]):.4f} Å⁻¹")

## 3. Ewald Shell Thickness

The Ewald sphere becomes a "shell" due to:
- **Energy spread** ΔE/E: causes Δk/k = ΔE/(2E)
- **Beam divergence** Δθ: causes Δk⊥ = k×Δθ

Combined in quadrature:
$$\sigma_{shell} = k \times \sqrt{\left(\frac{\Delta E}{2E}\right)^2 + \Delta\theta^2}$$

In [ ]:
# Calculate shell thickness for different voltages
voltages_kv = jnp.array([10, 15, 20, 25, 30])

print("Ewald shell thickness for different beam voltages:")
print("(ΔE/E = 10⁻⁴, Δθ = 1 mrad)")
print("=" * 45)

for V in voltages_kv:
    lam = rh.simul.wavelength_ang(V)
    k = 2 * jnp.pi / lam
    sigma_shell = rh.simul.compute_shell_sigma(
        k, energy_spread_frac=1e-4, beam_divergence_rad=1e-3
    )
    print(
        f"V = {int(V):2d} kV: λ = {float(lam):.4f} Å, k = {float(k):.1f} Å⁻¹, σ_shell = {float(sigma_shell):.4f} Å⁻¹"
    )

## 4. Rod-Ewald Overlap Visualization

The overlap factor determines how much intensity each reflection contributes. Let's visualize how it depends on domain size.

In [ ]:
# Build Ewald data for MgO
voltage_kv = 15.0
theta_deg = 2.0  # Grazing angle

ewald = rh.simul.build_ewald_data(
    crystal=crystal,
    voltage_kv=voltage_kv,
    hmax=3,
    kmax=3,
    lmax=2,
    temperature=300.0,
)

print(f"Electron wavelength: λ = {float(ewald.wavelength_ang):.4f} Å")
print(f"Wavevector magnitude: k = {float(ewald.k_magnitude):.2f} Å⁻¹")
print(f"Number of G vectors: {ewald.g_vectors.shape[0]}")

In [ ]:
# Compare overlap distributions for different domain sizes
domain_sizes_test = [20, 50, 100, 500]  # Å

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, L in zip(axes, domain_sizes_test):
    domain = jnp.array([float(L), float(L), float(L) / 2])
    overlap, intensities = rh.simul.finite_domain_intensities(
        ewald=ewald,
        theta_deg=theta_deg,
        phi_deg=0.0,
        domain_extent_ang=domain,
    )

    # Histogram of overlap values
    ax.hist(overlap, bins=50, range=(0, 1), alpha=0.7, edgecolor="black")
    ax.set_xlabel("Overlap factor", fontsize=11)
    ax.set_ylabel("Count", fontsize=11)
    ax.set_title(f"Domain size L = {L} Å", fontsize=12)

    # Statistics
    n_active = int(jnp.sum(overlap > 0.1))
    ax.axvline(x=0.1, color="r", linestyle="--", alpha=0.5)
    ax.text(
        0.95,
        0.95,
        f"Active (>0.1): {n_active}",
        transform=ax.transAxes,
        ha="right",
        va="top",
        fontsize=10,
    )

plt.suptitle("Overlap Factor Distribution vs Domain Size", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

**Observation**: Smaller domains have more "active" reflections (overlap > 0.1) because the broader rods intercept the Ewald shell over a wider range.

## 5. Intensity-Weighted Overlap

The measured diffraction pattern combines the base structure factor intensity with the overlap:

In [ ]:
# Compare intensity distributions
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Large domain (nearly infinite crystal)
domain_large = jnp.array([1000.0, 1000.0, 500.0])
overlap_large, intensities_large = rh.simul.finite_domain_intensities(
    ewald, theta_deg, 0.0, domain_large
)

# Small domain (polycrystalline)
domain_small = jnp.array([30.0, 30.0, 15.0])
overlap_small, intensities_small = rh.simul.finite_domain_intensities(
    ewald, theta_deg, 0.0, domain_small
)

# Plot base intensities
axes[0].semilogy(sorted(ewald.intensities, reverse=True), "b-", alpha=0.7)
axes[0].set_xlabel("Reflection rank")
axes[0].set_ylabel("Intensity (a.u.)")
axes[0].set_title("Base Structure Factor Intensities")
axes[0].grid(True, alpha=0.3)

# Plot large domain intensities
axes[1].semilogy(sorted(intensities_large, reverse=True), "g-", alpha=0.7)
axes[1].set_xlabel("Reflection rank")
axes[1].set_ylabel("Intensity (a.u.)")
axes[1].set_title(f"Large Domain (L = 1000 Å)")
axes[1].grid(True, alpha=0.3)

# Plot small domain intensities
axes[2].semilogy(sorted(intensities_small, reverse=True), "r-", alpha=0.7)
axes[2].set_xlabel("Reflection rank")
axes[2].set_ylabel("Intensity (a.u.)")
axes[2].set_title(f"Small Domain (L = 30 Å)")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print total intensity comparison
print(f"\nTotal intensity comparison:")
print(f"  Base intensities:     {float(jnp.sum(ewald.intensities)):.2e}")
print(f"  Large domain (1000Å): {float(jnp.sum(intensities_large)):.2e}")
print(f"  Small domain (30Å):   {float(jnp.sum(intensities_small)):.2e}")

## 6. Effect of Beam Parameters

Let's explore how beam quality affects the Ewald shell thickness and therefore the pattern.

In [ ]:
# Compare different beam conditions
beam_conditions = [
    ("High quality", 1e-5, 1e-4),  # FEG gun, small divergence
    ("Standard", 1e-4, 1e-3),  # Typical thermionic
    ("Poor quality", 1e-3, 5e-3),  # Large energy spread and divergence
]

k = ewald.k_magnitude
domain = jnp.array([100.0, 100.0, 50.0])  # Fixed domain size

print("Effect of beam quality on Ewald shell thickness:")
print("=" * 60)
print(
    f"{'Condition':<15} {'ΔE/E':<10} {'Δθ (mrad)':<12} {'σ_shell (Å⁻¹)':<15}"
)
print("-" * 60)

for name, dE_E, dtheta in beam_conditions:
    sigma_shell = rh.simul.compute_shell_sigma(k, dE_E, dtheta)
    print(
        f"{name:<15} {dE_E:<10.1e} {dtheta*1000:<12.2f} {float(sigma_shell):<15.4f}"
    )

print("\nFor comparison, rod width at L=100Å: σ_rod = 0.025 Å⁻¹")

In [ ]:
# Visualize overlap for different beam qualities
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, (name, dE_E, dtheta) in zip(axes, beam_conditions):
    overlap, intensities = rh.simul.finite_domain_intensities(
        ewald=ewald,
        theta_deg=theta_deg,
        phi_deg=0.0,
        domain_extent_ang=domain,
        energy_spread_frac=dE_E,
        beam_divergence_rad=dtheta,
    )

    ax.hist(overlap, bins=50, range=(0, 1), alpha=0.7, edgecolor="black")
    ax.set_xlabel("Overlap factor")
    ax.set_ylabel("Count")
    ax.set_title(f"{name}\n(ΔE/E={dE_E:.0e}, Δθ={dtheta*1000:.1f}mrad)")

    n_active = int(jnp.sum(overlap > 0.1))
    ax.text(
        0.95,
        0.95,
        f"Active: {n_active}",
        transform=ax.transAxes,
        ha="right",
        va="top",
    )

plt.suptitle(
    "Effect of Beam Quality on Overlap Distribution", fontsize=14, y=1.02
)
plt.tight_layout()
plt.show()

## 7. Summary

The finite domain broadening model accounts for two key effects:

1. **Rod broadening** from finite coherent domain size:
   - σ_rod = 2π/(L×√(2π)) ≈ 2.5/L Å⁻¹
   - Smaller domains → broader rods → more reflections contribute

2. **Shell broadening** from beam properties:
   - σ_shell = k×√[(ΔE/2E)² + Δθ²]
   - Typically dominated by beam divergence (≈1 mrad)

3. **Overlap integral** gives continuous intensity weighting:
   - Replaces binary "on/off" Ewald sphere condition
   - More realistic for imperfect crystals and real beams

### Typical Values

| Domain Size | Rod Width σ_rod |
|------------|----------------|
| 20 Å | 0.125 Å⁻¹ |
| 100 Å | 0.025 Å⁻¹ |
| 500 Å | 0.005 Å⁻¹ |

| Beam Quality | Shell Width σ_shell (at 15 kV) |
|-------------|-------------------------------|
| FEG, collimated | ~0.006 Å⁻¹ |
| Standard thermionic | ~0.06 Å⁻¹ |
| Poor | ~0.3 Å⁻¹ |

In [ ]:
# Quick reference table
print("Quick Reference: Rod Width vs Domain Size")
print("=" * 40)
for L in [10, 20, 50, 100, 200, 500, 1000]:
    extent = jnp.array([float(L), float(L), float(L)])
    sigma = rh.simul.extent_to_rod_sigma(extent)
    print(f"L = {L:4d} Å  →  σ = {float(sigma[0]):.4f} Å⁻¹")